# Openrouteservice: API de isócronas

In [1]:
import time
import json
import folium
import requests

### Consulta

In [2]:
ors_token = '<token>'
ors_url = 'https://api.openrouteservice.org/v2/isochrones'

# Perfis de rota
profile_urls = {
    'car': f'{ors_url}/driving-car',
    'foot': f'{ors_url}/foot-walking',
    'bike': f'{ors_url}/cycling-regular',
    'bus': f'{ors_url}/driving-hgv', # ônibus está incluído em HGV (Heavy Goods Vehicle)
}

coords_location = [-34.855469, -7.135231] # Comunidade São Rafael, João Pessoa-PB

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': ors_token,
    'Content-Type': 'application/json; charset=utf-8'
}

# Intervalos de tempo em segundos
list_range = [60*5, 60*10, 60*15, 60*20, 60*30]

result = {}

for profile, url in profile_urls.items():
    body = {'locations': [coords_location], 'range': list_range}
    response = requests.post(url, json=body, headers=headers)

    if response.ok:
        result[profile] = response.text
        print(f'Isócronas do perfil <{profile}>: sucesso!')
    else:
        print(f'Isócronas do perfil <{profile}>: falha -> {response.text}')
        result[profile] = None

    time.sleep(2)

Isócronas do perfil <car>: sucesso!
Isócronas do perfil <foot>: sucesso!
Isócronas do perfil <bike>: sucesso!
Isócronas do perfil <bus>: sucesso!


### Ajustes para visualização no mapa

In [3]:
m = folium.Map(location=coords_location[::-1], zoom_start=14, tiles='CartoDB dark_matter', control_scale=True)

list_colors = ['#ff0000', '#ffa700', '#fff400', '#a3ff00', '#2cba00']
classification = dict(zip(list_range, list_colors[::-1]))

def style_isochrones(feat):
    return {
        "color": classification[feat['properties']['value']],
        "opacity": 1,
        "fillOpacity": 0.3,
        "weight": 3,
        "dashArray": "6"
    }

for shape, geojson in result.items():
    geojson = json.loads(geojson)
    geojson['features'] = sorted(geojson['features'], key=lambda feat: feat['properties']['value'], reverse=True)
    geojson = json.dumps(geojson)
    folium.GeoJson(geojson, name=shape, style_function=style_isochrones).add_to(m)

marker = folium.CircleMarker(coords_location[::-1], radius=3, color='red', weight=6).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.keep_in_front(marker)

### Alguns resultados

In [4]:
m

In [5]:
m